In [ ]:
# Import library
from google.colab import files
from zipfile import ZipFile
import cv2

In [ ]:
# Upload Segmentation zip file
uploaded = files.upload()

Saving SegmentationNew.zip to SegmentationNew.zip


In [ ]:
# Specify zip file name
file_name = "SegmentationNew.zip"

# Extract dataset
with ZipFile(file_name, 'r') as zip:
    zip.printdir() # Print all files in the zip file
    print('Extracting all files...')
    zip.extractall("LeafSegmentationDisease") # Extracts all files into a new folder
    print('Dataset extracted successfully!')

File Name                                             Modified             Size
Segmentation/                                  2025-11-12 14:46:40            0
Segmentation/Training/                         2025-11-12 14:21:18            0
Segmentation/Training/Apple___Apple_scab/      2025-11-12 14:44:46            0
Segmentation/Training/Apple___Apple_scab/023123cb-7b69-4c9f-a521-766d7c8543bb___FREC_Scab 3487.JPG 2025-11-12 14:20:16        16337
Segmentation/Training/Apple___Apple_scab/1a304331-98b5-473f-bbb1-c33b8441052a___FREC_Scab 3296.JPG 2025-11-12 14:20:16        20612
Segmentation/Training/Apple___Apple_scab/1a41bab0-45e0-4dda-a798-9bf4a998f1b6___FREC_Scab 3450.JPG 2025-11-12 14:20:16        21636
Segmentation/Training/Apple___Apple_scab/1cb869ea-0a4c-47d7-9def-a88c16b72ddc___FREC_Scab 3350.JPG 2025-11-12 14:20:16        20607
Segmentation/Training/Apple___Apple_scab/1cdf2779-8cb1-4ae4-be70-b0eeb1abb0f8___FREC_Scab 3370.JPG 2025-11-12 14:20:16        20552
Segmentation/Trainin

In [ ]:
import os
# Setting constants
BATCH_SIZE = 32
Image_size = 224
input_shape = (Image_size, Image_size)
dataset_root = "/content/LeafSegmentationDisease/Segmentation"

train_dir = os.path.join(dataset_root, "Training")
test_dir = os.path.join(dataset_root, "Validation")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

# Define augmentations for train dataset and read the images
train_aug = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.30,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode="nearest",
)

# Read data from directory
train_data = train_aug.flow_from_directory(
    train_dir,
    target_size=(Image_size, Image_size),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

Found 1165 images belonging to 38 classes.


In [ ]:
# Augmentations for test data
test_aug = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# Read data from directory
test_data = test_aug.flow_from_directory(
    test_dir,
    target_size=(Image_size, Image_size),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

Found 494 images belonging to 38 classes.


In [ ]:
categories = list(train_data.class_indices.keys())
num_classes = len(categories)
print(f"Total categories: {num_classes}")
print(f"Categories: {categories}")

Total categories: 38
Categories: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D, Dropout


# Load the ResNet50 model
resnet_base = tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(Image_size,Image_size,3),
                   weights='imagenet')

resnet_base.trainable=True

for layer in resnet_base.layers[:-50]:
  layer.trainable = False

# Define the input layer for the model
inputs = Input(shape=(Image_size,Image_size,3))
x = resnet_base(inputs, training=False)

# Add custom classification layers
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(num_classes, activation='softmax')(x)


model = Model(inputs, outputs)
model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 38)             │         4,902 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,150,054 (92.13 MB)

 Trainable params: 17,512,870 (66.81 MB)

 Non-trainable params: 6,637,184 (25.32 MB)

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)


In [ ]:
history = model.fit(
  train_data,
  validation_data=test_data,
  epochs=25,
  callbacks=[early_stop]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
37/37 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.0467 - loss: 4.0110 - val_accuracy: 0.1377 - val_loss: 3.4101
Epoch 2/25
37/37 ━━━━━━━━━━━━━━━━━━━━ 20s 532ms/step - accuracy: 0.1323 - loss: 3.3358 - val_accuracy: 0.3785 - val_loss: 2.7985
Epoch 3/25
37/37 ━━━━━━━━━━━━━━━━━━━━ 19s 506ms/step - accuracy: 0.2090 - loss: 2.8797 - val_accuracy: 0.5425 - val_loss: 1.8759
Epoch 4/25
37/37 ━━━━━━━━━━━━━━━━━━━━ 19s 516ms/step - accuracy: 0.3424 - loss: 2.4253 - val_accuracy: 0.6660 - val_loss: 1.3125
Epoch 5/25
37/37 ━━━━━━━━━━━━━━━━━━━━ 19s 501ms/step - accuracy: 0.4573 - loss: 2.0025 - val_accuracy: 0.7389 - val_loss: 1.0165
Epoch 6/25
37/37 ━━━━━━━━━━━━━━━━━━━━ 19s 519ms/step - accuracy: 0.5764 - loss: 1.5763 - val_accuracy: 0.7834 - val_loss: 0.8402
Epoch 7/25
37/37 ━━━━━━━━━━━━━━━━━━━━ 19s 516ms/step - accuracy: 0.6180 - loss: 1.3855 - val_accuracy: 0.8239 - val_loss: 0.7384
Epoch 8/25
37/37 ━━━━━━━━━━━━━━━━━━━━ 19s 503ms/step - accuracy: 0.6766 - loss: 1.2039 - val_accurac

In [ ]:
model.save("LeafDiseaseResNet50.h5")

In [ ]:
from google.colab import files
files.download("/content/LeafDiseaseResNet50.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>